#**Task**

1. Formulate the above problem as a linear programming problem.

2. Implement the simplex method in Python to solve the problem.

3. Print the optimal solution (i.e., the values of the decision variables that maximize the profit and the corresponding maximum profit)

In [62]:
#Production quantities x1,x2,x2 of products A,B,C
#A,B,C use two raw materials X and Y
#p1,p2,p3 are profits from A,B,C products repectively
products = int(input("Enter no. of products : "))
materials = int(input("Enter no. of raw materials : "))
M = []
c = []
for i in range(materials):
  a = []
  for j in range(products):
    k = float(input("for product {}, coefficient {} : ".format(i+1, j+1)))
    a.append(k)
  M.append(a)
  d = float(input("constant of product {} : ".format(i+1)))
  c.append(d)
p = []
for i in range(products):
  e = float(input("prifit {} from product {} : ".format(i+1,i+1)))
  p.append(e)
print(M)
print(c)
print(p)

Enter no. of products : 2
Enter no. of raw materials : 3
for product 1, coefficient 1 : 2
for product 1, coefficient 2 : 1
constant of product 1 : 5
for product 2, coefficient 1 : 3
for product 2, coefficient 2 : -2
constant of product 2 : 6
for product 3, coefficient 1 : 1
for product 3, coefficient 2 : 4
constant of product 3 : 8
prifit 1 from product 1 : 3
prifit 2 from product 2 : 2
[[2.0, 1.0], [3.0, -2.0], [1.0, 4.0]]
[5.0, 6.0, 8.0]
[3.0, 2.0]


In [63]:
import numpy as np
import math

In [64]:
#adding slack variable in M
m = materials
M_slack = [[0.0] * m + [1.0 if i == j else 0.0 for j in range(m)] for i in range(m)]
for i in range(m):
    M_slack[i][:m] = M[i]

print(M_slack)

[[2.0, 1.0, 1.0, 0.0, 0.0], [3.0, -2.0, 0.0, 1.0, 0.0], [1.0, 4.0, 0.0, 0.0, 1.0]]


In [65]:
p = p + [0.0]*m
print(p)

[3.0, 2.0, 0.0, 0.0, 0.0]


In [66]:
M =  np.array(M_slack)
c = np.array(c)
p =  np.array(p)

In [67]:
#table where right most enteries are 'M',not include last row and last column
#last column enteries are c
#In last row n enteries are p and bottom right entry of table to 0
def tabular(p, M, c):
    m, n = M.shape
    table = np.zeros((m+1, n+1)) 
    table[:-1, :-1] = M
    table[:-1, -1] = c
    table[-1, :-1] = p
    table[-1, -1] = 0
    return table 

In [68]:
#function to return position of key_value
def key_value_position(table):
    z = table[-1][:-1]
    column  = np.argmax(np.array(z) > 0) #To create true value when z values are>0 and max value index store in column
    re = []
    for eq in table[:-1]:
        ele = eq[column]
        large = math.inf
        re.append(large if ele <= 0 else eq[-1] / ele) #when ratio<=0 it gives value to large so we can neglate that ratio
    row = re.index(min(re)) #remain ratio we find minimum of that and  give it to row index
    return row, column #return row and column index 

In [69]:
#function to take table andd a key position and return new table 
def key_step(table, key_position):
    new_table = [[] for eq in table]
    i, j = key_position
    key_value = table[i][j]
    new_table[i] = np.array(table[i]) / key_value  
    for eq_i, eq in enumerate(table):
        if eq_i != i:
            multiplier = np.array(new_table[i]) * table[eq_i][j]
            new_table[eq_i] = np.array(table[eq_i]) - multiplier  
    return new_table

In [70]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

In [71]:
#Final simplex solution
def simplex(p, M, c):
    table = tabular(p, M, c)
    
    while any(x > 0 for x in table[-1][:-1]): #need to be cj-zj<=0 for all,if not then go inside loop
        key_position = key_value_position(table)
        table = key_step(table,key_position)
    
    columns = np.array(table).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)  
    return solutions

In [72]:
solution = simplex(p, M, c)
nv = 0
for i in p:
  if i != 0:
     nv += 1
for i in range(nv):
  print("x{} : ".format(i+1),solution[i] )

x1 :  1.7142857142857146
x2 :  1.571428571428571


In [75]:
ans = 0
for i in range(nv):
  ans = ans + p[i]*solution[i]
print("Maximize of z : ",ans)

Maximize of z :  8.285714285714286
